In [2]:
# Import pandas for data manipulation
import pandas as pd

# Load the cleaned dataset saved from notebook 01
df = pd.read_csv("../data/processed/customer_segmentation_clean.csv")

# Show the first rows to confirm it loaded correctly
df.head()


,order_id,customer_id,order_date,product_category,product_name,quantity,unit_price,country,payment_method,sales_amount
0,O000001,C0103,2024-09-06,Accessories,Watch,3,153.48,UK,Bank Transfer,460.44
1,O000002,C0271,2024-07-05,Electronics,Keyboard,2,121.46,Spain,PayPal,242.92
2,O000003,C0107,2023-04-11,Home,Pillow,1,167.40,Italy,Bank Transfer,167.40
3,O000004,C0072,2023-12-25,Accessories,Watch,3,25.24,Germany,PayPal,75.72
4,O000005,C0189,2024-01-15,Accessories,Belt,3,100.60,Italy,Bank Transfer,301.80


In [3]:
# calculate RFM
# Import datetime tools to help calculate recency
from datetime import datetime
import pandas as pd

# Make sure order_date is in datetime format (safe step, even if it's already converted)
df["order_date"] = pd.to_datetime(df["order_date"])

# Set a reference date: one day after the last purchase in the dataset
reference_date = df["order_date"].max() + pd.Timedelta(days=1)

# Group by customer and calculate Recency, Frequency, Monetary
rfm = df.groupby("customer_id").agg(
    Recency = ("order_date", lambda x: (reference_date - x.max()).days),
    Frequency = ("order_id", "count"),
    Monetary = ("sales_amount", "sum")
)

# Look at the first rows of the RFM table
rfm.head()


,Recency,Frequency,Monetary
customer_id,,,
C0001,32,11,2946.80
C0002,105,7,2949.42
C0003,31,6,1850.35
C0004,90,8,2077.26
C0005,72,12,2756.33


In [4]:
# Assign scores for R, F, and M based on quantiles (1 = lowest, 5 = highest)

# Recency: lower recency = better (so scores are reversed)
rfm["R_score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1])

# Frequency: higher is better
rfm["F_score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])

# Monetary: higher is better
rfm["M_score"] = pd.qcut(rfm["Monetary"].rank(method="first"), 5, labels=[1,2,3,4,5])

# Create one combined score (e.g., 543, 215)
rfm["RFM_score"] = (
    rfm["R_score"].astype(int) * 100 +
    rfm["F_score"].astype(int) * 10 +
    rfm["M_score"].astype(int)
)

# Show the scored RFM table
rfm.head()


,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_score
customer_id,,,,,,,
C0001,32,11,2946.80,4,5,5,455
C0002,105,7,2949.42,2,3,5,235
C0003,31,6,1850.35,4,2,3,423
C0004,90,8,2077.26,2,4,4,244
C0005,72,12,2756.33,3,5,5,355


In [5]:
# Save the RFM table to the processed folder
rfm.to_csv("../data/processed/rfm_table.csv", index=False)

# Confirm save
"RFM table saved successfully"


'RFM table saved successfully'